## Check you have GPU attached

If you get error runing command. Go to `Edit > Notebook settings` and select "GPU".

In [ ]:
!nvidia-smi

## Download the lora file

I download the file from civitai gacha-splash-lora

https://civitai.com/models/13090/gacha-splash-lora

In [ ]:
!wget https://civitai.com/api/download/models/38884 --content-disposition

## Install requirements

In [ ]:
pip install safetensors diffusers transformers accelerate

## Add function to convert safetensor to diffusers

you cann't load safetensor file using `pipe.unet.load_attn_procs`, so convert to diffusers

In [ ]:
import torch
from diffusers import StableDiffusionPipeline


from safetensors.torch import load_file
from diffusers import StableDiffusionPipeline
import torch

def convert_lora_safetensor_to_diffusers(pipeline, checkpoint_path, LORA_PREFIX_UNET="lora_unet", LORA_PREFIX_TEXT_ENCODER="lora_te", alpha=0.75):
    # load LoRA weight from .safetensors
    state_dict = load_file(checkpoint_path)
    visited = []
    # directly update weight in diffusers model
    for key in state_dict:
        # it is suggested to print out the key, it usually will be something like below
        # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

        # as we have set the alpha beforehand, so just skip
        if ".alpha" in key or key in visited:
            continue

        if "text" in key:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
            curr_layer = pipeline.text_encoder
        else:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_UNET + "_")[-1].split("_")
            curr_layer = pipeline.unet

        # find the target layer
        temp_name = layer_infos.pop(0)
        while len(layer_infos) > -1:
            try:
                curr_layer = curr_layer.__getattr__(temp_name)
                if len(layer_infos) > 0:
                    temp_name = layer_infos.pop(0)
                elif len(layer_infos) == 0:
                    break
            except Exception:
                if len(temp_name) > 0:
                    temp_name += "_" + layer_infos.pop(0)
                else:
                    temp_name = layer_infos.pop(0)

        pair_keys = []
        if "lora_down" in key:
            pair_keys.append(key.replace("lora_down", "lora_up"))
            pair_keys.append(key)
        else:
            pair_keys.append(key)
            pair_keys.append(key.replace("lora_up", "lora_down"))

        # update weight
        if len(state_dict[pair_keys[0]].shape) == 4:
            weight_up = state_dict[pair_keys[0]].squeeze(3).squeeze(2).to(torch.float32)
            weight_down = state_dict[pair_keys[1]].squeeze(3).squeeze(2).to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down).unsqueeze(2).unsqueeze(3)
        else:
            weight_up = state_dict[pair_keys[0]].to(torch.float32)
            weight_down = state_dict[pair_keys[1]].to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down)

        # update visited list
        for item in pair_keys:
            visited.append(item)

    return pipeline

## Run the model

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_base = "Linaqruf/anything-v3.0"

pipe = StableDiffusionPipeline.from_pretrained(model_base, torch_dtype=torch.float16)

pipe = convert_lora_safetensor_to_diffusers(
    pipe,
    "./gachaSplashLORA_v40.safetensors",
)

pipe.to("cuda")

pipe.safety_checker = None if pipe.safety_checker is None else lambda images, **kwargs: (images, False)
pipe.enable_attention_slicing()

prompt = "[(white background:1.5),::5] 1girl, mid shot, full body, ocean, sea waves, water splashes, sky, light particles, butterflies, night, starry sky <lora:lihui4JXK-b2-bf16-128-128-1-re1-ep3-768-DA-5015fix:1>"
negative_prompt = "bad anatomy, extra fingers, watermark, (worst quality, low quality:1.4)"

image = pipe(
    prompt,
    height=728,
    width=512,
    num_inference_steps=24,
    guidance_scale=7.5,
    negative_prompt=negative_prompt,
).images[0]

image.save("./result.png")

In [ ]:
from PIL import Image

filename = "./result.png"

im = Image.open(filename)

im